In [1]:
# generating simple mesh

from mpi4py import MPI # message passing interface
from dolfinx import mesh # creation, refining and marking of meshes

domain = mesh.create_unit_square(MPI.COMM_WORLD, 8,8, mesh.CellType.quadrilateral)

# domain parameters:
# MPI - MPI-communicator. This is to specify how we would like the program to behave in parallel. 
# rest is obvious

In [2]:
# defining the finite element space

from dolfinx.fem import FunctionSpace

V = FunctionSpace(domain, ("CG", 1))

# second argument contains element type
# 'CG' - continuous lagrange
# 1 - element degree (liner here)


Next step is to specify the boundary condition $u = u_D$ on $\partial\Omega_D$

In [3]:

from dolfinx import fem

# defining uD as a function in function space V
uD = fem.Function(V)
uD.interpolate(lambda x: 1 + x[0] ** 2 + 2 * x[1] ** 2)

# applying the boundary values to all degrees of freedom 
# that are on the boundary of the discrete domain
# by identifying the facets (line-segments) representing the outer boundary

import numpy as np
# Create facet to cell connectivity required to determine boundary facets
tdim = domain.topology.dim # dimension of the mesh topology 
fdim = tdim - 1 # dimensionality of the facets (here - lines)
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

In [4]:
# finding the local indices of the degrees of freedom of the facets

boundary_dofs = fem.locate_dofs_topological(
    V,  # function space
    fdim,  # facet dimension
    boundary_facets,
)

# creating the Dirichlet boundary condition

bc = fem.dirichletbc(
    uD,
    boundary_dofs,
)


In [5]:
# The Unified Form Language (UFL) is a domain specific language
# for declaration of finite element discretizations of variational forms.
# More precisely, it defines a flexible interface for choosing finite element
# spaces and defining expressions for weak forms in a notation
# close to mathematical notation.
import ufl

# defining the trial and test function
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)


In [6]:
# PETSc - Portable, Extensible Toolkit for Scientific Computation
# petsc4py - Python bindings for PETSc
from petsc4py.PETSc import ScalarType

# Defining the source term
f = fem.Constant(domain, ScalarType(-6))
# fem.Constant - A constant with respect to a domain
# ScalarType - Double-precision floating-point number type, 
# compatible with Python `float` and C ``double``

In [7]:
# Defining the variational problem

# bilinear form a
a = ufl.dot(ufl.grad(u),ufl.grad(v)) * ufl.dx

#linear form L
L = f*v*ufl.dx

# ufl.dot - UFL operator: 
# Take the dot product of *a* and *b*. 
# This won't take the complex conjugate of the second argument

# ufl.grad -  UFL operator: 
# Take the gradient of *f*.
# For convention see documentation

In [8]:
# Forming and solving the linear system

problem = fem.petsc.LinearProblem(
    a,
    L,
    bcs=[bc],
    petsc_options={ # petsc as linear algebra backend, parameters
        "ksp_type": "preonly", # 
        "pc_type": "lu", #direct solver - LU factorization
    },
)

uh = problem.solve()

# https://petsc.org/main/docs/manual/ksp/?highlight=ksp#ksp-linear-system-solvers

In [9]:
# Computing the error

# interpolating the exact solution into P2 space
V2 = fem.FunctionSpace(domain, ("CG", 2))
# uex - u exact
uex = fem.Function(V2)
uex.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)

In [10]:
# computing the L2 norm of the error
diff = uh - uex
L2_error = fem.form(ufl.inner(diff, diff) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = np.sqrt(
    domain.comm.allreduce(
        error_local,
        op=MPI.SUM,
    )
)


In [11]:
# computing the maximum error at any degree of freedom

error_max = np.max(np.abs(uD.x.array - uh.x.array))

# Only print the error on one process
if domain.comm.rank == 0:
    print(f"Error_L2 : {error_L2:.2e}")
    print(f"Error_max : {error_max:.2e}")

Error_L2 : 8.24e-03
Error_max : 4.00e-15


In [12]:
from dolfinx import plot
import pyvista as pv

# dolfinx.plot - Support functions for plotting
# pyvista - an interface to VTK - scientific Visualisation Toolkit

#converting the mesh into the VTK format
topology, cell_types, geometry = plot.create_vtk_mesh(
    domain,
    tdim,
)

grid = pv.UnstructuredGrid(
    topology,
    cell_types,
    geometry,
)

In [13]:
pv.set_jupyter_backend("pythreejs")

In [14]:
#plotting the mesh

plotter = pv.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
plotter.show()

2022-08-14 14:20:07.264 (   0.790s) [        E407B000]    vtkExtractEdges.cxx:435   INFO| Executing edge extractor: points are renumbered
2022-08-14 14:20:07.264 (   0.791s) [        E407B000]    vtkExtractEdges.cxx:551   INFO| Created 144 edges


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [15]:
# plotting a function

# creating a mesh based on the DOF coordinates
u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(V)

u_grid = pv.UnstructuredGrid(
    u_topology,
    u_cell_types,
    u_geometry,)

u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pv.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
u_plotter.show()

2022-08-14 14:20:07.791 (   1.317s) [        E407B000]    vtkExtractEdges.cxx:435   INFO| Executing edge extractor: points are renumbered
2022-08-14 14:20:07.791 (   1.318s) [        E407B000]    vtkExtractEdges.cxx:551   INFO| Created 144 edges


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [16]:
# ipygany backend

warped = u_grid.warp_by_scalar()
plotter2 = pv.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
plotter2.show(jupyter_backend="ipygany")

AppLayout(children=(VBox(children=(HTML(value='<h3>u</h3>'), Dropdown(description='Colormap:', options={'BrBG'…

In [17]:
##############################################
#
#
#
# END OF TUTORIAL
#
#
#
##############################################

In [18]:
# Now applying the entire process to different geometries to reinforce the learning ^_^

In [19]:
def get_function_space(domain):
    return FunctionSpace(domain, ("CG", 1))


def get_boundary(V):
    uD = fem.Function(V)
    uD.interpolate(lambda x: 1 + x[0] ** 2 + 2 * x[1] ** 2)
    return uD


def get_boundary_facets(domain, fdim, tdim):
    domain.topology.create_connectivity(fdim, tdim)
    return mesh.exterior_facet_indices(domain.topology)


def compute_error(uh, uD):
    V2 = fem.FunctionSpace(domain, ("CG", 2))
    uex = fem.Function(V2)
    uex.interpolate(lambda x: 1 + x[0] ** 2 + 2 * x[1] ** 2)
    diff = uh - uex
    L2_error = fem.form(ufl.inner(diff, diff) * ufl.dx)
    error_local = fem.assemble_scalar(L2_error)
    error_L2 = np.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))
    error_max = np.max(np.abs(uD.x.array - uh.x.array))
    # Only print the error on one process
    if domain.comm.rank == 0:
        print(f"Error_L2 : {error_L2:.2e}")
        print(f"Error_max : {error_max:.2e}")


def solve_problem(domain):
    V = get_function_space(domain)
    uD = get_boundary(V)
    tdim = domain.topology.dim
    fdim = tdim - 1
    boundary_facets = get_boundary_facets(domain, fdim, tdim)
    boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
    bc = fem.dirichletbc(uD, boundary_dofs)
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)
    f = fem.Constant(domain, ScalarType(-6))
    a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
    L = f * v * ufl.dx
    problem = fem.petsc.LinearProblem(
        a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"}
    )
    uh = problem.solve()
    compute_error(uh, uD)
    return uh


def visualize_solution(domain, uh):
    V = get_function_space(domain)
    u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(V)
    u_grid = pv.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
    u_grid.point_data["u"] = uh.x.array.real
    u_grid.set_active_scalars("u")
    warped = u_grid.warp_by_scalar()
    u_plotter = pv.Plotter()
    u_plotter.add_mesh(warped, show_edges=True)
    u_plotter.view_xy()
    u_plotter.show(jupyter_backend="pythreejs")


In [23]:
domain = mesh.create_rectangle(
    MPI.COMM_WORLD, [[0, 0], [1,1]], [100, 100], mesh.CellType.quadrilateral
)
uh = solve_problem(domain)
visualize_solution(domain,uh)


Error_L2 : 5.27e-05
Error_max : 7.64e-14


2022-08-14 14:21:10.383 (  63.910s) [        E407B000]    vtkExtractEdges.cxx:435   INFO| Executing edge extractor: points are renumbered
2022-08-14 14:21:10.395 (  63.921s) [        E407B000]    vtkExtractEdges.cxx:551   INFO| Created 20200 edges


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [21]:
domain = mesh.create_rectangle(
    MPI.COMM_WORLD, [[0, 0], [1,1]], [10, 10], mesh.CellType.quadrilateral
)
uh = solve_problem(domain)


Error_L2 : 5.27e-03
Error_max : 3.11e-15
